In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from plyer import notification 
import requests
from tqdm import tqdm

In [2]:
def alerta(nivel, base, etapa, erro=""):

    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{erro}"


    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )

In [3]:
def tabelas_bd():

    conn = sqlite3.connect('coderhouse.db')

    # Executar uma consulta que retorna as informações do esquema do banco de dados
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()

    return schema



In [4]:
def salva_bd(df, nome_tabela):

    conn = sqlite3.connect('coderhouse.db')

    # Escrever o DataFrame na tabela 'nome_tabela'
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

    conn.close()

    return True



In [5]:
def carrega_bd(nome_tabela):

    conn = sqlite3.connect('coderhouse.db')

    # Executar uma consulta SELECT na tabela 'produtos' e converter em um DataFrame
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)

    conn.close()

    return df

In [6]:
def get_json_api(url):

    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        return data_json
    else:
        print("Erro no acesso a API")

In [7]:
def get_base_banks_url():

    next_url = "https://brasilapi.com.br/api/banks/v1"
    df_banks_full = pd.DataFrame()
    #while next_url is not None:
    #print(next_url, end='\r')
        
    json_data = get_json_api(next_url)

    df_banks = pd.DataFrame(json_data)

    df_banks_full = pd.concat([df_banks_full,df_banks],ignore_index=True)
    
        #next page
        #next_url = json_data['next']
    
    return df_banks_full

In [8]:
def get_base_banks():

    df_banks_url = carrega_bd("banks_url")

    df_banks_full = pd.DataFrame()

    #for url in tqdm(df_banks_url['url'].values):

        #GET
    json_data = get_json_api("https://brasilapi.com.br/api/banks/v1")


    pais = pd.DataFrame(json_data)
    pais.dtypes
    pais.info()
    pais = pais.fillna({'ispb':-1,'code':-1, 'name':'NO NAME', 'fullName':'NO NAME'})
    pais = pais.drop_duplicates(['ispb','code', 'name', 'fullName'])
    pais.shape
    pais
    pais = pais.pivot_table(index=('ispb'),
                 columns='code',
                 values='name',aggfunc="sum", fill_value=0)
    
    return pais
    

In [9]:
def etapa_extracao():
    
    #################################################
    print("  > Base banks_url:\n")
    try:
         df_banks_url = get_base_banks_url()
         salva_bd(df_banks_url,"banks_url")
    except Exception as e:
         alerta(nivel = 3, 
                base = "banks_url", 
                etapa = "EXTRACAO", 
                erro=e)
        
    #################################################
    print("  > Base banks:\n")
    try:
         df_banks_full = get_base_banks()
         salva_bd(df_banks_full,"banks")
    except Exception as e:
         alerta(nivel = 3, 
                base = "banks", 
                etapa = "EXTRACAO", 
                erro=e)

In [10]:
etapa_extracao()

  > Base banks_url:

  > Base banks:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359 entries, 0 to 358
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ispb      359 non-null    object 
 1   name      357 non-null    object 
 2   code      348 non-null    float64
 3   fullName  357 non-null    object 
dtypes: float64(1), object(3)
memory usage: 11.3+ KB


In [11]:
df = carrega_bd('banks')
df.head(30)

,-1.0,1.0,3.0,4.0,7.0,10.0,11.0,12.0,14.0,15.0,...,746.0,747.0,748.0,751.0,752.0,753.0,754.0,755.0,756.0,757.0
0,NO NAME,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,BCO DO BRASIL S.A.,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Selic,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bacen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
